In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, make_scorer
from sklearn.decomposition import PCA

In [2]:
path_archivo = "../input_data/combi_500_100.csv"
path_rand1 = "../input_data/uni_cp_1.csv"
path_rand2 = "../input_data/uni_cp_2.csv"
path_rand3 = "../input_data/uni_cp_3.csv"
path_rand4 = "../input_data/uni_cp_4.csv"
path_rand5 = "../input_data/uni_cp_5.csv"
datos_combi  = pd.read_csv(path_archivo)
datos_rand1 = pd.read_csv(path_rand1)
datos_rand2 = pd.read_csv(path_rand2)
datos_rand3 = pd.read_csv(path_rand3)
datos_rand4 = pd.read_csv(path_rand4)
datos_rand5 = pd.read_csv(path_rand5)

In [3]:
datos_combi = datos_combi.sort_values(by=["eta", "beta", "phi_K"]).reset_index(drop=True)

In [4]:
N_eig = 4
target = "phi_K"
def preprocesar_datos(d_frame, N_eig, target, opt = True):
    features_eig = list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
    features_dim = ["eta", "beta"]
    feature_especial = ["eig_0"]
    features_tot = [target] + feature_especial + features_dim + features_eig
    dat_copy = d_frame.copy()
    for i in range(N_eig):
        key_mod = "eig_" + str(i+1)
        prev_key = "eig_" + str(i)
        if i == 0:  
            dat_copy[key_mod] = 1/d_frame[key_mod] if opt else d_frame[prev_key]/d_frame[key_mod]
        else:
            dat_copy[key_mod] = d_frame[prev_key]/d_frame[key_mod]
        #fin if 
    #fin for
    for i in range(N_eig):
        col = "eig_" + str(i+1)
        #dat_copy[col] = np.log(dat_copy[col])
    #fin for 
    dat_copy = dat_copy.dropna()
    try:
        problematic_rows = dat_copy[(~np.isfinite(dat_copy)).any(axis=1) | (dat_copy.abs() > np.finfo(np.float64).max).any(axis=1)]
        problematic_indices = problematic_rows.index
        dat_copy = dat_copy.drop(index=problematic_indices)
    except:
        pass
    
    return dat_copy[features_tot]
#fin procesar_datos

In [5]:
datos_rand3

,phi_K,eta,beta,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,...,eig_13,eig_14,eig_15,eig_16,eig_17,eig_18,eig_19,Parallelepiped,Cylinder,Ellipsoid
0,0.074226,1.887066,2.596304,0.250610,1.867122,1.899091,2.249264,2.251893,2.711227,2.821912,...,3.876099,4.752189,4.932797,5.272763,5.277456,5.327887,5.577482,1,0,0
1,0.370823,1.729958,2.692846,1.156949,1.861579,2.057510,2.245613,2.297382,2.835063,2.894392,...,3.982999,4.994553,5.054116,5.298017,5.512737,5.522756,5.733596,1,0,0
2,0.292040,1.359813,0.329103,0.003327,1.723584,4.714291,6.107772,8.169655,15.353033,22.618414,...,83.708134,96.546230,109.439756,113.193161,124.114191,125.115624,174.196322,1,0,0
3,0.068775,1.358984,0.831063,0.010544,1.837741,5.406838,5.450475,7.325502,11.288224,13.334723,...,28.272695,33.122507,33.182899,34.257510,40.521839,41.024158,46.548213,1,0,0
4,0.331008,1.400031,3.015560,0.727970,1.514802,1.594106,2.953528,3.149933,3.346541,3.571850,...,6.365723,6.367226,6.724278,7.036735,7.369711,7.579719,7.800128,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.313115,0.430120,0.416537,0.000087,7.071457,8.186470,31.879775,34.128763,69.026692,82.503839,...,585.428394,829.031095,1082.222273,1241.362777,1312.630333,1788.313709,1934.814451,1,0,0
49996,0.523074,1.895420,0.844134,0.156092,1.467840,4.499118,4.675719,5.736605,6.595713,8.543306,...,20.030609,20.411442,21.242196,21.316892,24.570791,24.742701,25.972751,1,0,0
49997,1.039645,1.077865,2.155461,0.562235,1.325153,1.781364,3.913060,4.873205,5.245770,5.629269,...,12.601311,12.644962,14.883743,14.890805,15.282637,16.064852,18.550907,1,0,0
49998,0.015996,1.490191,0.522777,0.000850,2.199354,4.859238,6.480999,7.320386,20.792019,21.087767,...,36.924860,48.485613,66.246616,67.242809,68.047543,81.492253,84.628265,1,0,0


In [6]:
datos_combi = preprocesar_datos(datos_combi, N_eig, target)
datos_rand1 = preprocesar_datos(datos_rand1, N_eig, target)
datos_rand2 = preprocesar_datos(datos_rand2, N_eig, target)
datos_rand3 = preprocesar_datos(datos_rand3, N_eig, target)
datos_rand4 = preprocesar_datos(datos_rand4, N_eig, target)
datos_rand5 = preprocesar_datos(datos_rand5, N_eig, target)

In [7]:
datos_rand3

,phi_K,eig_0,eta,beta,eig_1,eig_2,eig_3,eig_4
0,0.074226,0.250610,1.887066,2.596304,0.535584,0.983166,0.844317,0.998833
1,0.370823,1.156949,1.729958,2.692846,0.537178,0.904773,0.916235,0.977466
2,0.292040,0.003327,1.359813,0.329103,0.580186,0.365608,0.771851,0.747617
3,0.068775,0.010544,1.358984,0.831063,0.544146,0.339892,0.991994,0.744041
4,0.331008,0.727970,1.400031,3.015560,0.660152,0.950252,0.539729,0.937648
...,...,...,...,...,...,...,...,...
49995,0.313115,0.000087,0.430120,0.416537,0.141414,0.863798,0.256792,0.934103
49996,0.523074,0.156092,1.895420,0.844134,0.681273,0.326251,0.962230,0.815067
49997,1.039645,0.562235,1.077865,2.155461,0.754630,0.743898,0.455235,0.802975
49998,0.015996,0.000850,1.490191,0.522777,0.454679,0.452613,0.749767,0.885336


In [8]:
metadata_temporal = {"combi": len(datos_combi), "rand1": len(datos_rand1), "rand2": len(datos_rand2)}
print(metadata_temporal)

{'combi': 52500, 'rand1': 49996, 'rand2': 49995}


In [9]:
N_phi_K = 500
N_datos = len(datos_combi)
N_partes = int(N_datos/N_phi_K)
print(N_partes)

105


In [10]:
def graficar_lambdas(data_frame, target, n_eig, preds = {}, op = lambda x: x, nombre = "lineal/", nombre_feat = "lambda_moño"):
    cols_eig = list(map(lambda x: "pca" + str(x), range(n_eig)))
    cols_preds = list(map(lambda x: "pred_" + x, cols_eig)) if len(preds) != 0 else []
    eta = str(np.array(data_frame["eta"])[0]/np.pi)[:4] + "pi"
    beta = str(np.array(data_frame["beta"])[0]/np.pi)[:4] + "pi"
    x = data_frame[target]
    ys = op(data_frame[cols_eig])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, ys)
    if len(preds) != 0:
        ax.plot(preds["y"], ys)
        nombre = "pred/" + nombre
    #fin if 
    ax.set_xlabel(target)
    ax.set_ylabel(nombre_feat)
    ax.set_title("eta=" +  eta + ", beta=" + beta)
    ax.legend(cols_eig + cols_preds)
    plt.savefig("pictures/" + nombre + "eta=" +  eta + ",beta=" + beta + "_" + ".png")
    plt.close()
#fin for 

In [11]:
def graficar_todas(d_frame, n_eig, target, op = lambda x: x, preds = {}, nombre = "lineal/", nombre_feat = "l_moño"):
    for n in range(N_partes):
        start = N_phi_K*n
        end = N_phi_K*(n+1)
        ys = preds["y"][start:end] if len(preds) != 0 else []
        preds_fin = {"y": ys, "X": preds["X"].iloc[start:end]} if len(preds) != 0 else {}
        graficar_lambdas(d_frame.iloc[start:end], target, n_eig, preds_fin, op, nombre, nombre_feat)
    #fin for 
#fin función

In [12]:
def get_custom_metrics(y, y_gorro):
    MAPE = (1/len(y))*sum(map(lambda x,z: abs((x-z)/x), y, y_gorro))
    MCPE = (1/len(y))*(sum(map(lambda x,z: ((x-z)/x)**2, y, y_gorro)))**0.5
    return {"MAPE": MAPE, "MCPE": MCPE}
#fin función

def get_metrics(X, y, model):
    y_gorro = model.predict(X)
    R2 = r2_score(y, y_gorro)
    RMSE = root_mean_squared_error(y, y_gorro)
    MAE = mean_absolute_error(y, y_gorro)
    #results = get_custom_metrics(y, y_gorro)
    return {"R2": R2, "RMSE": RMSE/(np.pi/2), "MAE": MAE/(np.pi/2)}
#fin función

class TransformadorDeDatos(BaseEstimator, TransformerMixin):
    def __init__(self, cols_tr, deg = 3):
        self.deg = deg
        self.cols_tr = cols_tr
    #fin init

    def fit(self, X, y=None):
        return self
    #fin fit

    def transform(self, X):
        datos = X.copy()
        cols_transformar = self.cols_tr
        poly = PolynomialFeatures(degree=self.deg, include_bias=False)
        poly_data = poly.fit_transform(datos[cols_transformar])
        poly_feature_names = poly.get_feature_names_out(input_features=cols_transformar)
        datos_finales = pd.DataFrame(poly_data, columns=poly_feature_names)
        return datos_finales
    #fin transform
#fin class

def crear_pipeline(modelo, grado, nombre_modelo, cols_tr):
    pipeline = Pipeline([('custom-transformer', TransformadorDeDatos(cols_tr, deg = grado)),
            #('power-transformer', PowerTransformer(method = "box-cox", standardize = True)), 
            (nombre_modelo, modelo)])
    return pipeline
#fin función

In [13]:
orden_pol = 4
features = ["eta", "beta"] + list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
pipeline = crear_pipeline(LinearRegression(), orden_pol, "re_lineal_simple", features)

In [14]:
X_combi = datos_combi[features]
X_rand1 = datos_rand1[features]
X_rand2 = datos_rand2[features]
X_rand3 = datos_rand3[features]
X_rand4 = datos_rand4[features]
X_rand5 = datos_rand5[features]
y_combi = datos_combi[target]
y_rand1 = datos_rand1[target]
y_rand2 = datos_rand2[target]
y_rand3 = datos_rand3[target]
y_rand4 = datos_rand4[target]
y_rand5 = datos_rand5[target]

In [15]:
X_train = pd.concat((X_rand2, X_rand3, X_rand4, X_rand5), axis = 0)
y_train = pd.concat((y_rand2, y_rand3, y_rand4, y_rand5), axis = 0)

In [16]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('custom-transformer',
                 TransformadorDeDatos(cols_tr=['eta', 'beta', 'eig_1', 'eig_2',
                                               'eig_3', 'eig_4'],
                                      deg=4)),
                ('re_lineal_simple', LinearRegression())])

In [17]:
#X_test1 = pd.concat((X_rand1, X_rand5), axis = 0)
#y_test1 = pd.concat((y_rand1, y_rand5), axis = 0)
metrs_train = get_metrics(X_train, y_train, pipeline)
metrs_test = get_metrics(X_rand1, y_rand1, pipeline)

In [18]:
print(metrs_train)
print(metrs_test)

{'R2': 0.8033817053284107, 'RMSE': np.float64(0.12815527702843532), 'MAE': np.float64(0.09319883688734099)}
{'R2': 0.5153147234485809, 'RMSE': np.float64(0.20133472701414626), 'MAE': np.float64(0.09437909952062135)}


In [19]:
path_archivo_kg = "../input_data/KG_random.csv"
str_to_ndarray = lambda x: np.fromstring(x, sep=' ')
datos_kg_random = pd.read_csv(path_archivo_kg, converters={'eigvals': str_to_ndarray})

In [20]:
datos_kg_random = datos_kg_random[datos_kg_random["shape"] == "parallelepiped"].copy()

In [21]:
del datos_kg_random["rho"]

In [22]:
datos_kg_random

,K,G,dx,dy,dz,shape,eigvals
98304,1.137564,0.925310,0.345791,0.722980,0.454774,parallelepiped,"[-1.5547410733937762e-13, -6.69847392884328e-1..."
98305,5.500412,5.321621,0.402060,0.999638,0.807374,parallelepiped,"[-2.1667717054444154e-12, -2.0543006395375965e..."
98306,3.375617,3.980662,0.429638,0.851665,0.529495,parallelepiped,"[-4.817232318550446e-13, -3.323750112763421e-1..."
98307,1.448420,4.074601,0.151318,0.942633,0.328977,parallelepiped,"[-2.6737693556924537e-12, -2.351597437975512e-..."
98308,0.351677,5.407791,0.446276,0.792512,0.989446,parallelepiped,"[-1.6759891759031827e-13, -3.9777497738663696e..."
...,...,...,...,...,...,...,...
131067,0.603325,3.002608,0.720661,0.439132,0.152189,parallelepiped,"[-2.9206283664231715e-12, -7.439679690463981e-..."
131068,2.302202,2.474801,0.132153,0.607512,0.605882,parallelepiped,"[-3.9535462097488675e-13, -1.163763376545102e-..."
131069,4.789391,3.091901,0.174914,0.402842,0.230977,parallelepiped,"[-5.3840655925037205e-12, -2.069314763230146e-..."
131070,0.920026,0.420177,0.185685,0.716565,0.322597,parallelepiped,"[-4.557497223449767e-13, -3.4356261396269377e-..."


In [23]:
for q in range(N_eig+1):#nondeg_minlen):
    datos_kg_random['eig_' + str(q)] = datos_kg_random['eigvals'].apply(lambda arr: arr[6 + q]) 

datos_kg_random = datos_kg_random.drop(columns=['eigvals'])

In [24]:
datos_kg_random.head()

,K,G,dx,dy,dz,shape,eig_0,eig_1,eig_2,eig_3,eig_4
98304,1.137564,0.925310,0.345791,0.722980,0.454774,parallelepiped,3.344897,4.253522,5.383594,10.070057,10.958655
98305,5.500412,5.321621,0.402060,0.999638,0.807374,parallelepiped,6.564257,10.320303,18.541555,20.077801,24.790483
98306,3.375617,3.980662,0.429638,0.851665,0.529495,parallelepiped,10.756621,12.849225,15.328886,29.027839,33.703187
98307,1.448420,4.074601,0.151318,0.942633,0.328977,parallelepiped,1.659110,5.366767,5.584904,10.405840,17.398203
98308,0.351677,5.407791,0.446276,0.792512,0.989446,parallelepiped,2.555409,4.226163,4.688969,7.011962,7.698858


In [25]:
datos_kg_random["phi_K"] = np.arctan(datos_kg_random["G"]/datos_kg_random["K"])
indexes_kg_random = [98304, 131071]
datos_kg_random["eta"] = np.ones(len(datos_kg_random))
datos_kg_random["beta"] = np.ones(len(datos_kg_random))
for i in range(indexes_kg_random[0], indexes_kg_random[1] + 1):
    lis_or = np.sort([datos_kg_random["dx"][i], datos_kg_random["dy"][i], datos_kg_random["dz"][i]])
    r = (lis_or[0]**2 + lis_or[1]**2 + lis_or[2]**2)**0.5
    datos_kg_random.loc[i, "eta"] = 2*np.arccos(lis_or[2]/r)
    datos_kg_random.loc[i, "beta"] = 4*np.arctan(lis_or[0]/lis_or[1])
#fin for 
datos_kg_random = datos_kg_random.copy()

In [26]:
datos_kg_random_copy = preprocesar_datos(datos_kg_random, N_eig, target, opt = False)

In [27]:
datos_kg_random_copy

,phi_K,eig_0,eta,beta,eig_1,eig_2,eig_3,eig_4
98304,0.682866,3.344897,1.337486,2.600392,0.786383,0.790090,0.534614,0.918914
98305,0.768879,6.564257,1.468138,1.848136,0.636053,0.556604,0.923485,0.809900
98306,0.867462,10.756621,1.350259,2.726644,0.837142,0.838236,0.528075,0.861279
98307,1.229252,1.659110,0.733530,1.724437,0.309145,0.960942,0.536709,0.598099
98308,1.505856,2.555409,1.486674,2.051426,0.604664,0.901299,0.668710,0.910779
...,...,...,...,...,...,...,...,...
131067,1.372503,2.463035,1.145569,1.334456,0.531536,0.528263,0.606136,0.973601
131068,0.821514,2.507252,1.591347,0.859013,0.474287,0.873008,0.471576,0.996856
131069,0.573257,36.155819,1.247024,2.592575,0.836996,0.772776,0.496226,0.869119
131070,0.428413,1.034104,0.958174,2.089119,0.812896,0.630602,0.401596,0.997046


In [28]:
X_test3 = datos_kg_random_copy[features]
y_test3 = datos_kg_random_copy[target]

In [29]:
metrs3 = get_metrics(X_test3, y_test3, pipeline)

In [30]:
print(metrs3)

{'R2': 0.7805019704677874, 'RMSE': np.float64(0.10951945179394772), 'MAE': np.float64(0.0843803853180547)}
